In [1]:
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib.request as req
import time
from urllib.request import urlopen
from scipy.stats import norm
import quandl


In [2]:
def get_historical_prices(symbol):
    site = 'https://api.iextrading.com/1.0/stock/' + symbol + '/chart/5y'
    data = json.loads(req.urlopen(site).read().decode())
    data = json.dumps(data)
    data = pd.read_json(data)
    data.index = pd.DatetimeIndex(data['date'])
    return data

def clean_options_df(df, spot_price, call_put_flag):
    df = df.drop(['contractSize', 'currency'], axis=1)
    df['expiration'] = pd.to_datetime(df['expiration'], unit='s')
    today = pd.to_datetime(pd.Timestamp.today().strftime('%D'))
    df['days to expiration'] = (df['expiration'] - today).dt.days
    df['B/A Spread'] = df['ask'] - df['bid']

    df = df.dropna()
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df['lastTradeDate'] = pd.to_datetime(df['lastTradeDate'], unit='s')
    df['Days Since Last Trade'] = -(df['lastTradeDate'] - today).dt.days
    df['Spot Price'] = spot_price
    df['Call_Put_Flag'] = call_put_flag
    return df

In [7]:
def optionsdata(ticker):
    site = 'https://query2.finance.yahoo.com/v7/finance/options/' + ticker
    with urlopen(site) as response:
        DATA = response.read().decode()
        DATA = json.loads(DATA)

    Expirations = DATA["optionChain"]['result'][0]['expirationDates']
    Expiration_Dates = pd.to_datetime(Expirations, unit='s')
    Strikes = pd.Series(DATA["optionChain"]['result'][0]['strikes'])
    strikes = DATA["optionChain"]['result'][0]['strikes']
    ###collecting stock data####
    index = []
    entry = []
    for a in DATA["optionChain"]['result'][0]['quote'].items():
        index.append(a[0])
        entry.append(a[1])

    todays_quotes = pd.DataFrame(entry, index=index, columns=['Entry'])
    Master_List = []
    Master_Calls = []
    Master_Puts = []
    for expiration in Expirations:
        try:

            expiry = str(expiration)
            expiration_date = pd.to_datetime(expiration, unit='s')
            new_site = 'https://query2.finance.yahoo.com/v7/finance/options/' + ticker + '?date=' + expiry

            with urlopen(new_site) as response:
                DATA = response.read().decode()
                DATA = json.loads(DATA)
                Master_List.append(DATA)
                Call_df = pd.read_json(json.dumps(DATA['optionChain']['result'][0]['options'][0]['calls']))
                Put_df = pd.read_json(json.dumps(DATA['optionChain']['result'][0]['options'][0]['puts']))
                Master_Calls.append(Call_df)
                Master_Puts.append(Put_df)
        except:
            Exception

    Calls = pd.concat(Master_Calls)
    Puts = pd.concat(Master_Puts)
    spot = todays_quotes.loc['regularMarketPrice'].iloc[0]

    # Clean Call Data
    Calls = clean_options_df(Calls, spot, 'Call')
    Puts = clean_options_df(Puts, spot, 'Put')
    Calls[['delta', 'gamma', 'vega', 'theta', 'rho']] = Calls.apply(greeks, axis=1)
    Puts[['delta', 'gamma', 'vega', 'theta', 'rho']] = Puts.apply(greeks, axis=1)
    Calls[['$/Delta', '$/Gamma']] = Calls.apply(per_dollar_basis, axis=1)
    Puts[['$/Delta', '$/Gamma']] = Puts.apply(per_dollar_basis, axis=1)

    return Calls, Puts

def per_dollar_basis(df):
    delta = df['delta']
    gamma = df['gamma']
    try:
        price = df['lastPrice']
    except:
        price = df['Ask']
    try:
        dol_per_delta = price / delta
        dol_per_gamma = price / gamma
    except:
        dol_per_delta = np.nan
        dol_per_gamma = np.nan

    return pd.Series([dol_per_delta, dol_per_gamma])


def greeks(df):
    rate = 0.015
    q = 0
    spot = df['Spot Price']
    strike = df['strike']
    vol = df['impliedVolatility']
    maturity = df['days to expiration']
    Call_Put_Flag = df['Call_Put_Flag']

    """ask, bid, change, contractSymbol, expiration,vol, inTheMoney, lastPrice, lastTradeDate,
    openInterest, percentChange, strike, volume,maturity, BA_Spread, DaysSinceLastTrade,
    spot, Call_Put_Flag = list(column_list)"""
    d1 = (np.log(spot / strike) + (maturity / 365) * (rate - q + (vol ** 2) / 2)) / (vol * np.sqrt(maturity / 365))
    d2 = d1 - vol * np.sqrt(maturity / 365)
    gamma = ((np.exp(-q * maturity / 365) / (spot * vol * np.sqrt(maturity / 365))) * 1 / np.sqrt(2 * np.pi)) * np.exp(
        (-d1 ** 2) / 2)
    vega = (((spot * np.exp(-q * maturity / 365)) * np.sqrt(maturity / 365)) / 100) * 1 / np.sqrt(2 * 3.14159) * np.exp(
        (-d1 ** 2) / 2)

    if Call_Put_Flag == 'Call':
        try:
            delta = np.exp(-q * maturity / 365) * norm.cdf(d1)
        except:
            delta= np.nan
        try:
            theta = ((1 / (maturity / 365)) * -((spot * vol * np.exp(-q * maturity / 365))) / 2 * np.sqrt(
            maturity / 365)) * (np.exp((-d1 ** 2) / 2)) / np.sqrt(2 * 3.14159) - rate * strike * np.exp(
            -rate * maturity / 365) * norm.cdf(d2) + q * spot * np.exp(rate * maturity / 365) * norm.cdf(d1)
        except:
            theta = np.nan
        try:
            rho = strike * maturity / 365 * np.exp(-rate * maturity / 365) * norm.cdf(d2)
        except:
            rho = np.nan

    if Call_Put_Flag == 'Put':
        try:
            delta = np.exp(-rate * maturity / 365) * (norm.cdf(d1) - 1)
        except:
            delta = np.nan
        try:
            theta = ((1 / maturity / 365) * -(spot * vol * np.exp(-q * maturity / 365)) / 2 * np.sqrt(maturity / 365) * (
            np.exp((-d1 ** 2) / 2)) / np.sqrt(2 * 3.14159)) + rate * strike * np.exp(-rate * maturity / 365) * norm.cdf(
            -d2) - q * spot * np.exp(rate * maturity / 365) * norm.cdf(-d1)
        except:
            theta = np.nan
        try:

            rho = -strike * maturity / 365 * np.exp(-rate * maturity / 365) * norm.cdf(-d2)
        except:
            rho = np.nan

    return pd.Series([delta, gamma, vega, theta, rho])




In [8]:
calls,puts=optionsdata('TSLA')

In [9]:
calls

,ask,bid,change,contractSymbol,expiration,impliedVolatility,inTheMoney,lastPrice,lastTradeDate,openInterest,...,Days Since Last Trade,Spot Price,Call_Put_Flag,delta,gamma,vega,theta,rho,$/Delta,$/Gamma
0,142.60,141.20,13.539993,TSLA190222C00150000,2019-02-22,0.000010,True,150.20,2019-02-21 14:35:09,2,...,0,292.74,Call,1.000000,0.000000,0.000000,-2.249908,0.410942,NaN,NaN
1,132.65,131.25,0.000000,TSLA190222C00160000,2019-02-22,0.000010,True,126.88,2019-01-23 19:00:51,2,...,29,292.74,Call,1.000000,0.000000,0.000000,-2.399901,0.438338,NaN,NaN
2,92.70,91.25,-1.000000,TSLA190222C00200000,2019-02-22,0.000010,True,101.00,2019-02-21 15:01:45,12,...,0,292.74,Call,1.000000,0.000000,0.000000,-2.999877,0.547923,NaN,NaN
3,72.55,71.25,0.000000,TSLA190222C00220000,2019-02-22,0.000010,True,87.50,2019-02-15 20:28:23,3,...,6,292.74,Call,1.000000,0.000000,0.000000,-3.299864,0.602715,NaN,NaN
4,67.60,66.25,0.000000,TSLA190222C00225000,2019-02-22,0.000010,True,87.40,2019-02-11 14:43:43,1,...,10,292.74,Call,1.000000,0.000000,0.000000,-3.374861,0.616413,NaN,NaN
5,62.10,61.00,0.000000,TSLA190222C00230000,2019-02-22,0.000010,True,70.48,2019-02-20 17:38:18,8,...,1,292.74,Call,1.000000,0.000000,0.000000,-3.449858,0.630111,NaN,NaN
6,57.50,56.10,0.000000,TSLA190222C00235000,2019-02-22,0.000010,True,70.99,2019-02-20 14:40:46,17,...,1,292.74,Call,1.000000,0.000000,0.000000,-3.524855,0.643809,NaN,NaN
7,47.60,46.30,0.000000,TSLA190222C00245000,2019-02-22,0.000010,True,49.90,2019-01-23 17:24:18,1,...,29,292.74,Call,1.000000,0.000000,0.000000,-3.674849,0.671205,NaN,NaN
8,42.25,41.30,-12.899998,TSLA190222C00250000,2019-02-22,0.000010,True,45.61,2019-02-21 18:23:04,40,...,0,292.74,Call,1.000000,0.000000,0.000000,-3.749846,0.684903,NaN,NaN
9,40.05,39.10,0.000000,TSLA190222C00252500,2019-02-22,0.000010,True,45.70,2019-01-24 16:49:48,5,...,28,292.74,Call,1.000000,0.000000,0.000000,-3.787344,0.691752,NaN,NaN
